In [1]:
#!/usr/bin/env python

# T500_1900-2010.ipynb

In [2]:
'''
    File name: T500_1900-2010.ipynb
    Author: Andreas Prein
    E-mail: prein@ucar.edu
    Date created: 23.07.2020
    Date last modified: 23.07.2020

    ##############################################################
    Purpos:

    1) Reads in 3-hourly T500 and convert it to daily means

    2) store the data as NetCDF


'''

'\n    File name: T500_1900-2010.ipynb\n    Author: Andreas Prein\n    E-mail: prein@ucar.edu\n    Date created: 23.07.2020\n    Date last modified: 23.07.2020\n\n    ##############################################################\n    Purpos:\n\n    1) Reads in 3-hourly T500 and convert it to daily means\n\n    2) store the data as NetCDF\n\n\n'

In [ ]:
from dateutil import rrule
import datetime
import glob
from netCDF4 import Dataset
import sys, traceback
import dateutil.parser as dparser
import string
from pdb import set_trace as stop
import numpy as np
import numpy.ma as ma
import os
from mpl_toolkits import basemap
import pickle
import subprocess
import pandas as pd
from scipy import stats
import copy
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib as mpl
import pylab as plt
import random
import scipy.ndimage as ndimage
import matplotlib.gridspec as gridspec
from mpl_toolkits.basemap import Basemap, cm
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.gridspec as gridspec
from pylab import *
import string
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
import shapefile
import shapely.geometry
# import descartes
import shapefile
import math
from scipy.stats.kde import gaussian_kde
from math import radians, cos, sin, asin, sqrt
from scipy import spatial
import scipy.ndimage
import matplotlib.path as mplPath
from scipy.interpolate import interp1d
import time
from math import atan2, degrees, pi
import scipy
import scipy.ndimage as ndimage
import scipy.ndimage.filters as filters
# import SkewT
import csv
import pygrib
from scipy import interpolate

In [9]:
# ________________________________________________________________________
# ________________________________________________________________________
#             USER INPUT SECTION

DataDir='/gpfs/fs1/collections/rda/data/ds131.3/anl/'

Plevs=['500']# ['10', 850','500','200']
PLEVSall=np.array([1000, 975, 950, 925, 900, 850, 800, 750, 700, 650, 600, 550, 500, 450, 400, 350, 300, 250, 200, 150, 100, 70, 50, 30, 20, 10, 5, 1])
iLev = np.where(int(Plevs[0]) == PLEVSall)[0][0]
ConstantVars='/gpfs/fs1/collections/rda/data/ds131.3/anl/anl_mean_2015_VGRD_pres.nc'
FinVar='T500'
SaveDir='/glade/campaign/mmm/c3we/prein/NCEP-20C/'+FinVar+'/daymean'

rgdTime = pd.date_range(datetime.datetime(1836, 1, 1,0), end=datetime.datetime(2015, 12, 31,23), freq='d')
YYYYall = np.unique(rgdTime.year)

In [11]:
# ________________________________________________________________________
# ________________________________________________________________________
#              READ IN THE COORDINATES
ncid=Dataset(ConstantVars, mode='r')
rgrLat=np.squeeze(ncid.variables['latitude'][:])
rgrLon=np.squeeze(ncid.variables['longitude'][:])
ncid.close()

In [18]:
for yy in range(len(YYYYall)):
    print('    process '+str(YYYYall[yy]))
    U_file=DataDir+'anl_mean_'+str(YYYYall[yy])+'_TMP_pres.nc'
    Uvar='t'

    T_Filename=U_file.split('/')[-1]
    File_fin=SaveDir+'/'+T_Filename.replace('Uvar', FinVar)
    File=SaveDir+T_Filename.replace(Uvar, FinVar)+'_copy'
    if os.path.exists(File_fin) == 0:
        if not os.path.exists(SaveDir):
            os.makedirs(SaveDir)
        print( '    '+U_file)
        ncid=Dataset(U_file, mode='r')
        rgrU=np.squeeze(ncid.variables[Uvar][:,iLev,:,:])
        time_act=np.squeeze(ncid.variables['time'][:])[4::8]
        ncid.close()
        try:
            U10 = np.min(np.reshape(rgrU, (int(rgrU.shape[0]/8),8,rgrU.shape[1],rgrU.shape[2])), axis=1)
        except:
            U10 = U10[-int(U10.shape[0]/8)*8:,:]
            U10 = np.min(np.reshape(U10, (int(U10.shape[0]/8),8,U10.shape[1],U10.shape[2])), axis=1)

        # ________________________________________________________________________
        # write the netcdf
        print( '        ----------------------')
        print( '        Save data to '+File_fin)
        root_grp = Dataset(File, 'w', format='NETCDF4')
        # dimensions
        root_grp.createDimension('time', None)
        root_grp.createDimension('rlon', rgrLon.shape[0])
        root_grp.createDimension('rlat', rgrLat.shape[0])
        # variables
        lat = root_grp.createVariable('lat', 'f4', ('rlat',))
        lon = root_grp.createVariable('lon', 'f4', ('rlon',))
        time = root_grp.createVariable('time', 'f8', ('time',))
        UV = root_grp.createVariable('T2M', 'f4', ('time','rlat','rlon',),fill_value=-99999)

        time.calendar = "proleptic_gregorian"
        time.units = "hours since 1800-01-01 00:00:00.0"
        time.standard_name = "time"
        time.long_name = "time"

        lon.standard_name = "longitude"
        lon.long_name = "longitude"
        lon.units = "degrees_east"

        lat.standard_name = "latitude"
        lat.long_name = "latitude"
        lat.units = "degrees_north"

        UV.standard_name = "temperature at 500 hPa"
        UV.long_name = "temperature at 500 hPa"
        UV.units = "K"

        # write data to netcdf
        lat[:]=rgrLat
        lon[:]=rgrLon
        UV[:]=U10
        time[:]=time_act
        root_grp.close()

        # compress the netcdf file
        pp = subprocess.Popen("nccopy -k 4 -d 1 -s "+File+' '+File_fin, shell=True)
        pp.wait()
        subprocess.Popen("rm  "+File, shell=True)

    process 1836
    process 1837
    process 1838
    process 1839
    process 1840
    process 1841
    process 1842
    process 1843
    process 1844
    process 1845
    process 1846
    process 1847
    process 1848
    process 1849
    process 1850
    process 1851
    process 1852
    process 1853
    process 1854
    process 1855
    process 1856
    process 1857
    process 1858
    process 1859
    process 1860
    process 1861
    process 1862
    process 1863
    process 1864
    process 1865
    process 1866
    process 1867
    process 1868
    process 1869
    process 1870
    process 1871
    process 1872
    process 1873
    process 1874
    process 1875
    process 1876
    process 1877
    process 1878
    process 1879
    process 1880
    process 1881
    process 1882
    process 1883
    process 1884
    process 1885
    process 1886
    process 1887
    process 1888
    process 1889
    process 1890
    process 1891
    process 1892
    process 1893
    process 18